In [1]:
# Mount the Google Drive to Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import the libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from PIL import Image
import os, random, shutil

In [3]:
# Specify some parameters
learning_rate = 1e-4
batch_size = 16
epochs_number = 100

In [ ]:
# Define the path to the dataset
dataset_path = '/content/drive/MyDrive/Colab Notebooks/RNSA_Subset_PNGs_12K'

# Get a list of all the folders in the dataset directory
folders = [os.path.join(dataset_path, folder_name) for folder_name in os.listdir(dataset_path)]

# Initialize empty lists to store the file paths for each set
train_set, validation_set, test_set = [], [], []

# Loop over each folder and split the PNG images into sets
for folder in folders:
    # Get the folder name (label)
    folder_name = os.path.basename(folder)
    # Get a list of all the PNG images in the folder
    png_images = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.png')]
    random.shuffle(png_images)
    # Define the percentage split for train, validation, and test sets
    train_percent, validation_percent, test_percent = 0.75, 0.15, 0.10
    # Split the PNG images into train, validation, and test sets
    num_files = len(png_images)
    train_end = int(num_files * train_percent)
    validation_end = int(num_files * (train_percent + validation_percent))
    train_set += png_images[:train_end]
    validation_set += png_images[train_end:validation_end]
    test_set += png_images[validation_end:]


# Create train, validation, and testing directories
for directory in ['Train', 'Validation', 'Test']:
    os.makedirs(os.path.join(dataset_path, directory), exist_ok=True)


# Loop over the train, validation, and test sets, and copy each image to the appropriate directory
for set_name, set_files in [('Train', train_set), ('Validation', validation_set), ('Test', test_set)]:
    for folder_name in ['Normal', 'Any','EDH', 'SDH', 'SAH', 'IPH', 'IVH']:
        # Create the folder in the set directory
        os.makedirs(os.path.join(dataset_path, set_name, folder_name), exist_ok=True)
        # Get the images in the current folder
        folder_files = [f for f in set_files if folder_name in f]
        # Copy the images to the set directory
        for f in folder_files:
            dest_folder = os.path.join(dataset_path, set_name, folder_name)
            shutil.copy(f, dest_folder)

# Print the number of images in each set
print("Train set size:", len(train_set))
print("Validation set size:", len(validation_set))
print("Test set size:", len(test_set))

Train set size: 5679
Validation set size: 1135
Test set size: 758


In [4]:
# Define the ZFNet architecture
model = tf.keras.models.Sequential([
    # Convolutional layer 1
    Conv2D(filters=96, kernel_size=(7, 7), strides=(2, 2), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    # Convolutional layer 2
    Conv2D(filters=256, kernel_size=(5, 5), strides=(2, 2), activation='relu'),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    # Convolutional layer 3
    Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    # Convolutional layer 4
    Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    # Convolutional layer 5
    Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    # Flatten the output from convolutional layers
    Flatten(),
    # Fully-connected layer 1
    Dense(units=4096, activation='relu'),
    # Fully-connected layer 2
    Dense(units=4096, activation='relu'),
    # Output layer with 7 classes
    Dense(units=7, activation='softmax')
])

In [5]:
# Compile the model
model.compile(optimizer = Adam(learning_rate = learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

# View the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 125, 125, 96)      14208     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 384)       1

In [6]:
# Dataset Normalization
train_datagen = ImageDataGenerator(rescale = 1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

# train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range = 20, horizontal_flip = True)

In [7]:
# Load the dataset
train_data = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/RNSA_Subset_PNGs_12K/Train', 
                                               target_size = (256, 256), 
                                               batch_size = batch_size, 
                                               class_mode = 'categorical')
validation_data = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/RNSA_Subset_PNGs_12K/Validation', 
                                                    target_size = (256, 256), 
                                                    batch_size = batch_size, 
                                                    class_mode = 'categorical')
#test_data = test_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/RNSA_Subset_PNGs_12K/Test', 
                                              #target_size = (256, 256), 
                                              #batch_size = batch_size, 
                                              #class_mode = 'categorical')

Found 5679 images belonging to 7 classes.
Found 1135 images belonging to 7 classes.


In [8]:
# Fit the model
res_history = model.fit(train_data, 
                        validation_data = validation_data, 
                        epochs = epochs_number, 
                        steps_per_epoch = len(train_data), 
                        validation_steps = len(validation_data))

Epoch 1/100
355/355 [==============================] - 36s 75ms/step - loss: 1.8825 - accuracy: 0.2210 - val_loss: 1.8381 - val_accuracy: 0.2529
Epoch 2/100
355/355 [==============================] - 25s 70ms/step - loss: 1.8212 - accuracy: 0.2500 - val_loss: 1.7995 - val_accuracy: 0.2581
Epoch 3/100
355/355 [==============================] - 27s 75ms/step - loss: 1.7779 - accuracy: 0.2823 - val_loss: 1.7669 - val_accuracy: 0.2811
Epoch 4/100
355/355 [==============================] - 26s 74ms/step - loss: 1.7224 - accuracy: 0.3108 - val_loss: 1.7532 - val_accuracy: 0.2943
Epoch 5/100
355/355 [==============================] - 25s 71ms/step - loss: 1.6745 - accuracy: 0.3372 - val_loss: 1.7483 - val_accuracy: 0.3075
Epoch 6/100
355/355 [==============================] - 25s 71ms/step - loss: 1.6193 - accuracy: 0.3599 - val_loss: 1.7161 - val_accuracy: 0.3436
Epoch 7/100
355/355 [==============================] - 26s 72ms/step - loss: 1.5508 - accuracy: 0.3969 - val_loss: 1.7312 - val_ac

In [ ]:
# Plot the loss
plt.plot(res_history.history['loss'], label='Train Loss')
plt.plot(res_history.history['val_loss'], label='Validation Loss')
plt.grid(True)
plt.legend()
plt.axes().set(xlabel='Epoch', ylabel='Loss', title='Architecture Loss');
plt.show()
plt.savefig('LossVal_loss')

# Plot the accuracy
plt.plot(res_history.history['accuracy'], label='Train Accuracy')
plt.plot(res_history.history['val_accuracy'], label='Validation Accuracy')
plt.grid(True)
plt.legend()
plt.axes().set(xlabel='Epoch', ylabel='Accuracy', title='Architecture Accuracy');
plt.show()
plt.savefig('AccVal_acc')